In [1]:
!pip install symusic mido pretty_midi miditoolkit music21

In [2]:
!wget https://github.com/lzqlzzq/minimidi/raw/main/example/mahler.mid

--2023-11-26 16:29:28--  https://github.com/lzqlzzq/minimidi/raw/main/example/mahler.mid
Resolving github.com (github.com)... 20.29.134.23
Connecting to github.com (github.com)|20.29.134.23|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lzqlzzq/minimidi/main/example/mahler.mid [following]
--2023-11-26 16:29:28--  https://raw.githubusercontent.com/lzqlzzq/minimidi/main/example/mahler.mid
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 478769 (468K) [audio/midi]
Saving to: ‘mahler.mid.4’

mahler.mid.4        100%[===================>] 467.55K  --.-KB/s    in 0.03s   

2023-11-26 16:29:28 (15.4 MB/s) - ‘mahler.mid.4’ saved [478769/478769]



# Load MIDI File

* print each objects in symusic could get the corresponding summaries
* Time unit is a quarter.
* A *pyi* file have been generated, you can browse the members and functions of each class in it.

In [3]:
from symusic import Score
score = Score("mahler.mid")
print(score)
print("note_num: ", score.note_num())
print("start_time: ", score.start_time())
print("end_time: ", score.end_time())


<Score track_num=16, note_num=35816, end_time=3554.000000>
note_num:  35816
start_time:  3.0
end_time:  3554.0


In [4]:
print(score.tempos)
print(score.key_signatures)
print(score.time_signatures)

<TempoList length=3333>
<KeySignatureList length=1>
<TimeSignatureList length=44>


In [5]:
print(score.tempos[0])
print(score.key_signatures[0])
print(score.time_signatures[0])

<Tempo time=0.000000, qpm=107.999916>
C
<TimeSignature time=0.000000, numerator=8, denominator=4>


In [6]:
print(score.tracks)
print(score.tracks[0])
print("track name: ", score.tracks[0].name)
print("is_drum: ", score.tracks[0].is_drum)
print("note_num: ", score.tracks[0].note_num())
print(score.tracks[0].notes)
print(score.tracks[0].notes[0])

<TrackList length=16>
<Track name="flute/picc", program=73, is_drum=False, note_num=2490, end_time=3554.000000>
track name:  flute/picc
is_drum:  False
note_num:  2490
<NoteList length=2490>
<Note start=533.000000, dur=2.000000, pitch=77, vel=100>


In [7]:
note = score.tracks[0].notes[0]
print("start:\t\t", note.start)
print("duration:\t", note.duration)
print("pitch:\t\t", note.pitch)
print("velocity:\t", note.velocity)

start:		 533.0
duration:	 2.0
pitch:		 77
velocity:	 100


# Bach Processing

* sort (in place)
* shift_time
* shift_pitch
* shift_velocity
* clip(start: float, end: float, clip_end: bool)
* filter_notes(func: Callable)
* note_array

In [8]:
# inplace operation
print(score.sort())
print(score.tracks[0].sort())

<Score track_num=16, note_num=35816, end_time=3554.000000>
<Track name="flute/picc", program=73, is_drum=False, note_num=2490, end_time=3554.000000>


In [9]:
# method chaining
score.shift_time(10) \
     .shift_pitch(-10) \
     .shift_velocity(10) \
     .sort() \
     .copy() # deepcopy

<Score track_num=16, note_num=0, end_time=0.000000>

In [10]:
score.clip(10, 100, True) # start: float, end: float, clip_end: bool

<Score track_num=16, note_num=476, end_time=99.852081>

In [11]:
score.filter_notes(lambda note: note.pitch > 60)

<Score track_num=16, note_num=17300, end_time=3549.500000>

In [12]:
note_arr = score.tracks[0].note_array()
print(note_arr)
print(type(note_arr.pitch), note_arr.pitch)
print(type(note_arr.velocity), note_arr.velocity)
print(type(note_arr.start), note_arr.start)
print(type(note_arr.duration), note_arr.duration)

<NoteArray name="flute/picc", program=73, note_num=2490>
<class 'numpy.ndarray'> [ 77  82  70 ... 101  23  24]
<class 'numpy.ndarray'> [100 100 100 ... 100 100 100]
<class 'numpy.ndarray'> [ 533.  533.  533. ... 3549. 3553. 3553.]
<class 'numpy.ndarray'> [2.  2.  2.  ... 0.5 1.  1. ]


In [13]:
frame_pianoroll = score.tracks[0].frame_pianoroll(quantization=24)
print("frame_pianoroll", frame_pianoroll.dtype, frame_pianoroll.shape)
onset_pianoroll = score.tracks[0].onset_pianoroll(quantization=16)
print("onset_pianoroll", onset_pianoroll.dtype, onset_pianoroll.shape)

frame_pianoroll bool (128, 21324)
onset_pianoroll bool (128, 14216)


# Benchmark

## MIDI Parsing

* mido is writen in pure python, and only parsing midi file to event level
* pretty_midi and miditoolkit is based on mido

In [14]:
# install julia for testing MIDI.jl
!pip install jill
!jill install -c True

JILL - Julia Installer 4 Linux (MacOS, Windows and FreeBSD) -- Light

querying release information from https://julialang-s3.julialang.org/bin/versions.json
julia 1.9.4 already installed.
True


In [15]:
import mido, music21
import pretty_midi as pm
import miditoolkit as mtk
p = "mahler.mid"

In [16]:
# %%timeit
# midi_jl.load(p)

In [17]:
%%timeit
Score(p)

23.3 ms ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%%timeit
# mido is writen in pure python, and only parsing midi file to event level
mido.MidiFile(p)

7.15 s ± 2.11 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
pm.PrettyMIDI(p)

7.1 s ± 1.64 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
mtk.MidiFile(p)

6.73 s ± 1.54 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit
music21.converter.parse(p)

10.2 s ± 558 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
jl_script = f"""
import Pkg
Pkg.add("MIDI")
Pkg.add("BenchmarkTools")
using BenchmarkTools
using MIDI
b = @benchmark load("{p}")
println("first run: ", b)
b = @benchmark load("{p}")
println("scecond run: ", b)
"""
with open("bench.jl", "w") as f:
  f.write(jl_script)

In [23]:
!julia bench.jl

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
first run: Trial(132.627 ms)
scecond run: Trial(132.829 ms)


## Bench Processing


In [24]:
score = Score(p)
score2 = score.copy()

In [25]:
%%timeit
score.copy()

276 µs ± 16.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [26]:
%%timeit
score.shift_pitch(10)

364 µs ± 6.46 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
%%timeit
score2.filter_notes(lambda note: note.pitch > 60)

31.6 ms ± 348 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%%timeit
[t.note_array() for t in score.tracks]

226 µs ± 8.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [29]:
%%timeit
[(t.onset_pianoroll(), t.frame_pianoroll()) for t in score.tracks]

10.3 ms ± 389 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
